# Colourizing Black and White Images Delivery

In this document, I will create an **user friendly app** for my chosen neural network(**NNW version 5** see the predictions document). For this I will use **Anvil**, Anvil is a free Python-based drag-and-drop web app builder that is able to get your trained AI working as an application.


created by: Mickey krekels
Class: AI45

## Loading the imports 

In [1]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from PIL import Image
import base64
import keras
import cv2
import os

from PIL import Image 
from skimage.io import imread, imshow ,imsave
from PIL import Image 
import base64
import requests
import json

import warnings
warnings.filterwarnings('ignore')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import anvil.server

anvil.server.connect("HE4D7DFHJOOA2QE3RPYKKS7U-LHCSCT2MSTF2AD3H")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


## Loading the trained encoder and decoder Model

In [3]:
encoder_Model =  tf.keras.models.load_model('Saves/tensorflow_Encoder_V1_ComputationTime_Network_v4'
                                           ,custom_objects=None,compile=True)

In [4]:
decoder_Model = tf.keras.models.load_model('Saves/tensorflow_V1_ComputationTime_Network_v4'
                                           ,custom_objects=None,compile=True)

PredictImage is a function that takes an image as an input parameter, it saves the size of the image and scales it down to the shape of the input of the network. After the predictions the image gets scaled back up to size, I extract the original light layer and combine it with the predicted A-B colour spectrums. This will result in an image with colour and no quality loss.

In [5]:
def PredictImage(image):    
    
    # extracting the inputs for the encoder and decoder network
    test = img_to_array(image)
    (H_orig,W_orig) = test.shape[:2] # original image size
    resized = resize(test, (224,224), anti_aliasing=True)
    resized*= 1.0/255
    lab = rgb2lab(resized)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    
    # predicting the the colors in the image
    vggpred = encoder_Model.predict(L)  
    ab      = decoder_Model.predict(vggpred) 

    # bringing the new AB map back to original size
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    Predicted_Image = lab2rgb(cur)
    Predicted_Image = cv2.resize(Predicted_Image , (W_orig,H_orig)) 
    
    # combining the orginal light layer with the predicted AB color spectrums 
    test*= 1.0/255
    col = rgb2lab(Predicted_Image)
    lig = rgb2lab(test)
    l  =  lig[:,:,0]
    ab =  col[:,:,1:] 
    
    # creating the final predicted image
    cur = np.zeros((H_orig, W_orig, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    Predicted_Image = lab2rgb(cur)
    
    return Predicted_Image

---
## Anvil

Creating a Connection with anvil servers and making a callable function to predict the image.

In [6]:
import anvil.media 

@anvil.server.callable
def colorize_image(file):
    path = 'data/output/output.png'
    
    with anvil.media.TempFile(file) as filename:
        img = cv2.imread(filename)
    
    input_img  = img
    data = PredictImage(input_img)
    imsave(path,data)

    
    with open(path, "rb") as file:
        url = "https://api.imgbb.com/1/upload"
        payload = {
            "key": 'ea029ff50b124083cfe7e27f9dcac6c8',
            "image": base64.b64encode(file.read()),
            "name" : "coloredImage",
        }
        res = requests.post(url, payload)
        
    jsondata = res.json()
    
    return str(jsondata['data']['url'])

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


---
# Final Conclusion 

Due to the way Anvil works returning an image as output and showing it on screen is not possible in the free version. I temporarily fixed this by uploading the image to a separate server, this allows me to get the URL image from the server when the picture is created. This version is for demonstration purposes only.  

But it works perfectly the AI can predict images in a user-friendly way(see **demo video below**). This document concludes this project, I learned a lot about using: **neural networks** , **Support Vector Machines** and **digital pictures** in general. In short, it was an interesting and fun project to work on!
 
[demo video link](https://youtu.be/SjLtwPhC1_w)
